### 수행목표
- KMRD 데이터를 기반으로 랜덤으로 추천하는 기능을 개발한다.

### 수행단계
- 데이터를 load() 하고 전처리하는 class는 MovieDataLoader로 하고 생성 시 입력 파라미터는 file 경로로 하여 로드 후 결과로 데이터를 return 한다.
- 추천 코드 class 명을 RandomRecommender로 한다.
- 데이터 파일은 평점(rates.csv)과 영화(movies.txt)만 사용한다.
- 추천을 실행하는 함수는 run() 으로 하고 실행 완료 후 결과에 random 평점 field(rate_random)를 추가하고 random 값을 반영해서 결과를 return 한다.
결과 데이터 평점 높은 순 n개를 출력한다.

In [17]:
### 랜덤 추천 작동 방식
# 1) 랜덤 평점 생성 - 0~10 사이의 무작위 값을 생성하여 `rate_random` 필드 추가  
# 2) 영화와 평점 데이터 병합 - `movie` 키를 기준으로 두 데이터 병합  
# 3) 랜덤 평점 기준 정렬 - `rate_random` 값을 기준으로 내림차순 정렬  
# 4) 상위 N개 영화 반환 - 정렬된 데이터에서 상위 N개 추출  

import pandas as pd
# import random # numpy의 np.random.uniform을 사용하므로 주석 처리
import numpy as np

In [18]:
# 데이터를 load() 하고 전처리하는 class
class MovieDataLoader:
    def __init__(self, movies_file: str, rates_file: str):
        # 영화 데이터 파일 경로와 평점 데이터 파일 경로를 저장
        self.movies_file = movies_file
        self.rates_file = rates_file

    def load(self):
        # 영화 데이터(movies.txt) 파일을 로드
        # '\t'로 구분된 데이터를 Pandas DataFrame으로 읽어옴
        # 'movie' 컬럼을 인덱스로 설정
        movies = pd.read_csv(self.movies_file, sep="\t", engine="python", index_col="movie")
        
        # 평점 데이터(rates.csv) 파일을 로드
        rates = pd.read_csv(self.rates_file)
        
        # 로드된 데이터프레임 반환
        return movies, rates

In [20]:
# 추천 코드 class
class RandomRecommender:
    def __init__(self, movies, rates):
        # 생성자에서 영화 데이터와 평점 데이터를 받아서 저장
        self.movies = movies
        self.rates = rates

    def run(self, n=10):

        # 방법 1) Pandas와 기본 파이썬 라이브러리(random)를 사용한 방식
        # 평점 데이터에 'rate_random' 컬럼 추가
        # 각 평점에 대해 0~10 사이의 무작위 값을 생성하여 저장
        # self.rates['rate_random'] = self.rates['rate'].apply(lambda _: random.uniform(0, 10))
        

        # 방법 2) Numpy의 **numpy.random.uniform**을 활용하여 랜덤 값을 생성하면, 
        # pandas 의 apply를 사용하지 않고 배열 전체를 벡터화 연산으로 처리할 수 있어 속도가 더 빠름
        # Numpy를 활용해 랜덤 평점(rate_random) 생성

        # random.uniform 은 소수점 16자리까지 생성할 수 있는 부동소수점 값을 반환하므로 
        # np.round 를 활용하여 소수점 둘째자리까지 반올림
        self.rates['rate_random'] = np.round(np.random.uniform(0, 10, size=len(self.rates)), 2)

        
        # 영화 데이터와 평점 데이터를 병합
        # 평점 데이터의 'movie' 컬럼과 영화 데이터의 인덱스를 기준으로 병합
        merged_data = pd.merge(self.rates, self.movies, left_on='movie', right_index=True)
        
        # 'rate_random' 컬럼을 기준으로 내림차순 정렬
        sorted_data = merged_data.sort_values(by="rate_random", ascending=False)
        
        # 상위 n개의 데이터 반환
        return sorted_data.head(n)

In [42]:
# 메인 실행부
if __name__ == "__main__":
    # 영화 데이터 파일 경로
    movies_file = "kmrd-small/movies.txt"
    # 평점 데이터 파일 경로
    rates_file = "kmrd-small/rates.csv"

    # 데이터 로드
    loader = MovieDataLoader(movies_file, rates_file)
    movies, rates = loader.load()

    # 랜덤 추천 실행
    recommender = RandomRecommender(movies, rates)
    # 상위 5개의 추천 결과 가져오기
    top_recommendations = recommender.run(n=5)
    # 추천 결과 출력
    print(top_recommendations)


    # 만약 결과 데이터 가공이 필요한 경우라면 `[[]]` 으로 필요한 컬럼을 선택하면 됨
    # DataFrame에서 여러 컬럼을 선택할 때는 [[...]]를 사용, 한개의 컬럼만 선택할 경우 [] 만 써도 되긴 함
    # 선택된 컬럼으로 새로운 DataFrame을 반환하므로, 원본 DataFrame은 영향을 받지 않음
    print()
    print()
    print()
    print("-------------------------") 
    print("-----결과 데이터 가공-----") 
    print("-------------------------") 

    result = top_recommendations[['rate_random', 'movie', 'rate', 'title', 'year']]

    print(result)

        user  movie  rate        time  rate_random         title  \
81569  12984  10200    10  1110803400         10.0  터미네이터 2:오리지널   
20049   1309  10038    10  1151815740         10.0        에이리언 2   
37274   3246  10389    10  1385012520         10.0           양철북   
48559   4946  10971     9  1129307220         10.0         블루 스틸   
18985   1189  10001    10  1261772640         10.0        시네마 천국   

                                    title_eng    year     grade  
81569       Terminator 2: Judgment Day , 1991  2019.0   15세 관람가  
20049                           Aliens , 1986  1986.0   15세 관람가  
37274  Die Blechtrommel , The Tin Drum , 1979  1988.0  청소년 관람불가  
48559                       Blue Steel , 1990  1990.0  청소년 관람불가  
18985                  Cinema Paradiso , 1988  2013.0    전체 관람가  



-------------------------
-----결과 데이터 가공-----
-------------------------
       rate_random  movie  rate         title    year
81569         10.0  10200    10  터미네이터 2:오리지널  2019.0
20049       